In [2]:
import numpy as np
#from sklearn.datasets import fetch_mldata
#from sklearn.model_selection import train_test_split
from scipy.signal import convolve2d, convolve
import torch
from torch import nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
# from torchvision.datasets import MNIST
# from torchvision import transforms
from IPython.display import clear_output
from typing import Tuple
%matplotlib inline

/home/sibwa19/documents/main_course/second/project/MusicTranscription/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
vel = np.array([
    [3, 2, 5, 1],
    [1, 2, 0, 3]
]).astype(float)

pitch = np.array([
    [3, -1, -1, -1],
    [1, 0, -1, -1]
]).astype(float)

# vel[pitch != -1].mean()
# pitch[pitch != -1]
pitch != -1


array([[ True, False, False, False],
       [ True,  True, False, False]])

In [6]:
pred = np.array([
    [0.1, 0.2, 0.5, 0.6],
    [0.25, 0.15, 0.65, 0.6]
])
y = np.array([
    [3, -1, -1, -1],
    [1, 0, -1, -1]
])

def batch_accuracy(X: np.ndarray, y: np.ndarray):
    accuracy = []
    cnt = 0
    success = 0

    X = np.argsort(pred, axis=1)[::, ::-1]
    y_cp = np.copy(y)

    for i in np.arange(X.shape[0]):
        break_point = y_cp[i].tolist().index(-1)
        for j in np.arange(break_point):
            if y_cp[i][j] in X[i][:break_point]:
                success += 1
            cnt += 1
    
    return success / cnt

lst = batch_accuracy(pred, y)
lst

0.3333333333333333

In [4]:
def train(network, epochs: int, learning_rate=1e-4):
    loss_nll = nn.NLLLoss()
    loss_multi = nn.MultiLabelMarginLoss()
    # loss_mse = nn.MSELoss()
    
    optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)
    train_loss_epochs = []
    test_loss_epochs = []
    train_accuracy_epochs = []
    test_accuracy_epochs = []
    
    try:
        for epoch in range(epochs):

            # train 
            losses_pitch = []
            losses_vel = []
            accuracies_pitch = []
            accuracies_vel = []
            for X, y in train_loader:
                network.zero_grad()
                # prediction = network.forward(X)
                pred_pitch, pred_vel = network.forward(X)

                loss_batch_pitch = loss_multi(pred_pitch, y[0])
                losses_pitch.append(loss_batch_pitch.item())
                loss_batch_pitch.backward()

                loss_batch_vel = loss_nll(pred_vel, y[1])
                losses_vel.append(loss_batch_vel.item())
                loss_batch_vel.backward()

                optimizer.step()
                accuracies_pitch.append(batch_accuracy(pred_pitch, y[0]))
                # left count vel
                accuracies_vel.append()

            train_loss_epochs.append(np.mean(losses_pitch))
            train_accuracy_epochs.append(np.mean(accuracies))

            # test
            losses = []
            accuracies = []    
            for X, y in test_loader:
                prediction = network.forward(X)
                loss_batch = loss_nll(prediction, y)
                losses.append(loss_batch.item())
                accuracies.append((np.argmax(prediction.data.numpy(), 1)==y.data.numpy()).mean())
            test_loss_epochs.append(np.mean(losses))
            test_accuracy_epochs.append(np.mean(accuracies))
            
            # output
            clear_output(True)
            print('\rEpoch {0}... (Train/Test) NLL: {1:.3f}/{2:.3f}\tAccuracy: {3:.3f}/{4:.3f}'.format(
                        epoch, train_loss_epochs[-1], test_loss_epochs[-1],
                        train_accuracy_epochs[-1], test_accuracy_epochs[-1]))
            plt.figure(figsize=(12, 5))
            plt.subplot(1, 2, 1)
            plt.plot(train_loss_epochs, label='Train')
            plt.plot(test_loss_epochs, label='Test')
            plt.xlabel('Epochs', fontsize=16)
            plt.ylabel('Loss', fontsize=16)
            plt.legend(loc=0, fontsize=16)
            plt.grid()
            plt.subplot(1, 2, 2)
            plt.plot(train_accuracy_epochs, label='Train accuracy')
            plt.plot(test_accuracy_epochs, label='Test accuracy')
            plt.xlabel('Epochs', fontsize=16)
            plt.ylabel('Loss', fontsize=16)
            plt.legend(loc=0, fontsize=16)
            plt.grid()
            plt.show()
    except KeyboardInterrupt:
        pass

### Conv2D

In [5]:
#constants
image_size = (28, 28)
conv1 = 32
conv2 = 64

# conv_nn
class ConvTwoD(nn.Module):
    def __init__(self, image_size: Tuple[int, int],
        conv1=conv1, conv2=conv2
    ):
        super(ConvTwoD, self).__init__()
        self.conv_layers1 = nn.Sequential(
            nn.Conv2d(1, conv1, kernel_size=(5,5), stride=(1,1)),
            nn.Tanh(),
            nn.Dropout(0.5),
            nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        )
        
        self.conv_layers2 = nn.Sequential(
            nn.Conv2d(conv1, conv2, kernel_size=(3,3), stride=(2,2)),
            nn.Tanh(),
            nn.Dropout(0.5),
            nn.MaxPool2d(kernel_size=(2,2))
        )
        
        self.linear_velocity = nn.Sequential(
            nn.Linear(
                conv2 * image_size[0]//2 * image_size[1]//2,
                128
            ),
            nn.LogSoftmax(dim=1)
        )

        self.linear_pitch = nn.Sequential(
            nn.Linear(
                conv2 * image_size[0]//2 * image_size[1]//2,
                128
            ),
            nn.LogSoftmax(dim=1)
        )
    
    def forward(self, x):
#         print("======")
#         print(x.shape)
        x = self.conv_layers1(x)
#         print(x.shape)
        x = self.conv_layers2(x)
#         print(x.shape)
        x = x.view(x.size(0), -1)
#         print(x.shape)
        pitch = self.linear_pitch(x)
        velocity = self.linear_velocity(x)
        
        return pitch, velocity

### Conv1D

In [6]:
#constants
image_size = 28
conv1 = 32
conv2 = 64

# conv_nn
class ConvOneD(nn.Module):
    def __init__(self, image_size: Tuple[int, int],
        conv1=conv1, conv2=conv2
    ):
        super(ConvOneD, self).__init__()
        self.conv_layers1 = nn.Sequential(
            # nn.Conv2d(1, conv1, kernel_size=(5,5), stride=(1,1)),
            nn.Tanh(),
            nn.Dropout(0.5),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv_layers2 = nn.Sequential(
            # nn.Conv2d(conv1, conv2, kernel_size=(3,3), stride=(2,2)),
            nn.Tanh(),
            nn.Dropout(0.5),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.linear_velocity = nn.Sequential(
            nn.Linear(
                conv2 * image_size,
                128
            ),
            nn.LogSoftmax(dim=1)
        )

        self.linear_pitch = nn.Sequential(
            nn.Linear(
                conv2 * image_size,
                128
            ),
            nn.LogSoftmax(dim=1)
        )
    
    def forward(self, x):
#         print("======")
#         print(x.shape)
        x = convolve(x, 5, mode="same")
        x = self.conv_layers1(x)
#         print(x.shape)
        x = convolve(x, 5, mode="same")
        # x = self.conv_layers2(x)
#         print(x.shape)
        x = x.view(x.size(0), -1)
#         print(x.shape)
        pitch = self.linear_pitch(x)
        velocity = self.linear_velocity(x)
        
        return pitch, velocity

In [ ]:
network1 = Conv(image_size=image_size)
train(network1, 5)